In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import nltk
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re #import the regular expression library
import string
import os
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import MiniBatchKMeans
from sklearn.datasets import load_files
from nltk import pos_tag
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
from nltk.corpus import stopwords #import the stopwords from the ntlk.corpus library
nltk.download('stopwords')
from nltk.tokenize import word_tokenize #import the word_tokenize method, which is used to turn sentences into words"
nltk.download('punkt')
from collections import Counter
from nltk.stem import PorterStemmer 



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [56]:
FullPaper=[]
directories = ["biorxiv_medrxiv"]
for directory in directories: #for each of the three folders carrying the json format of different research papers
    for file in tqdm(os.listdir(f"{directory}/{directory}/pdf_json")): #for every json file
        file_path=f"{directory}/{directory}/pdf_json/{file}" #set the file path to the file_path variable 
        paper = json.load(open(file_path,"rb"))
        title = paper['metadata']['title'] 
        try:
            abstract = paper['abstract']
        except:
            abstarct=""                
        full_text=""     
        
        for text in paper['body_text']:
            full_text +=text['text'] +'\n\n' 
        FullPaper.append([title,abstract,full_text])
        
FullPaperDataframe=pd.DataFrame(FullPaper,columns=['title','abstract','full_text'])

100%|██████████| 1934/1934 [01:12<00:00, 26.51it/s]


> Our aim is to address these points:
* Capabilities to discover a therapeutic (not vaccine) for the disease, and clinical effectiveness studies to discover therapeutics, to include antiviral agents.

In [78]:
full_text = FullPaperDataframe['full_text'].head(30)
clean_text = [] #a list which will hold all the bodies of the papers after being stripped out of stopwords
ps=PorterStemmer() 
# punctuation_regex = string.punctuation
# punctuation_regex = punctuation_regex.replace("-", "") # keep the hyphens
# pattern = r"[{}]".format(punctuation_regex) # generate the regex pattern
pattern = """!"#$%&'()*+,.:;<=>?@[\]^_`{|}~""" #the pattern which will account for punctuation, and will later be used to remove them
#cleaning the data and removing stop words
for val in  tqdm(full_text):
        body_tokens = word_tokenize(val)
        paper_body_without_stopwords = [token for token in body_tokens if not token in stopwords.words('english')] #remove the stop words in the body and return a list
        clean_string = ' '.join(paper_body_without_stopwords).lower() #convert the list into string
        clean_string = re.sub(rf"[{pattern}]", '', clean_string)#remove punctuation except for hyphens
        clean_string = re.sub(r"\bthe\b", r"", clean_string) #remove the
        clean_string = re.sub(r'\bwe\b', '',clean_string ) #remove the pronoun we
        clean_string = re.sub(r'\bit\b', '',clean_string )#remove the pronoun it
        clean_string = re.sub(r'\bthey\b', '',clean_string )#remove the pronoun they
        clean_string = re.sub(r'\bcopyright\b', '',clean_string )#remove the word copyright
        clean_string = re.sub(r'\bet\b', '',clean_string )#remove the word et
        clean_string = re.sub(r'\bal\b', '',clean_string )#remove the word al
        clean_string = re.sub(r'\bin\b', '',clean_string )#remove the preposition in
        clean_string = re.sub(r'\d+', '',clean_string ) #remove digits
        clean_text.append(clean_string) #add the string to the list 
stemmed_text=[] # a list which will contain the stemmed bodies of all docs      
for val in clean_text: #for the body (which was stripped out of stop words) of each paper
    stemned_val="" #create an empty string
    for word in val: #for each word in the body
        stemned_val+=ps.stem(word) #stem theword and concat it to the string stemned_val
    stemmed_text.append(stemned_val) #add the string stemned_val to the list which will contain the stemmed bodies of all docs  
text = ' '.join(stemmed_text) #convert all the stemmed bodies into one string containing all the stemmed bodies of all papers 
s=text.split(' ')#split on empty string
s_without_empty_strings = [string for string in s if string != ""]#a list containing all the stemmed words excluding the empty strings
r = Counter(s_without_empty_strings)# count the number of vocab (unique words)
most_occur = r.most_common(50) #get the most common 50 words
print(most_occur) #print the most common 50 words

# # finding the tf-idf matrix for all the abstracts
# # vec = TfidfVectorizer()
# # ve = vec.fit_transform(clean_abs)

# # # displaying the tf-idf of the word in the abstact  
# # pd.DataFrame(ve.toarray(), columns=sorted(vec.vocabulary_.keys()))

# # #applying tf-idf cosine similarity and printing the result(testing)
# # query = ["prophylaxis"]
# # query_tfidf = vec.transform(query)
# # cosineSimilarities = cosine_similarity(query_tfidf, ve).flatten()
# # print(cosineSimilarities)
    


100%|██████████| 30/30 [01:21<00:00,  2.72s/it]


[('preprint', 721), ('data', 395), ('model', 316), ('holder', 313), ('fig', 313), ('author/funder', 307), ('number', 305), ('license', 290), ('peer-reviewed', 288), ('https', 288), ('s', 277), ('medrxiv', 264), ('//doiorg//', 255), ('doi', 255), ('rna', 221), ('also', 221), ('covid-', 221), ('using', 218), ('cases', 216), ('epidemic', 215), ('available', 213), ('used', 196), ('virus', 193), ('rate', 188), ('one', 187), ('a', 184), ('two', 165), ('without', 162), ('this', 162), ('display', 159), ('made', 156), ('time', 156), ('international', 155), ('granted', 155), ('all', 154), ('perpetuity', 154), ('c', 150), ('may', 148), ('different', 147), ('figure', 144), ('biorxiv', 144), ('cells', 144), ('table', 143), ('countries', 142), ('analysis', 140), ('total', 138), ('reads', 137), ('-', 134), ('n', 134), ('would', 133)]


In [ ]:
taged_abs=[]
nouns=[]
# tagging the clean data
for val in clean_abs:
    taged_abs.append(pos_tag(val.split()))
# extracting the nouns
for i in range(len(taged_abs)):
    doc=[]
    for j in range(len(taged_abs[i])):
        if(taged_abs[i][j][1]=='NN' or taged_abs[i][j][1]=='NNS'):
            doc.append(taged_abs[i][j][0])
    nouns.append(doc)
        
print(nouns)



Above we dcided to calculate the tf-idf so that we can represent every word that is present in the abstract quatitavily. By doing so we can further use the results in order model the topic according to the abstract that we just quatified.Furthermore we will use (Non-negative Matrix Factorization) NMF in order to come up with topic's that carry most weight in the abstract. To accomplish this we are going to filter all the nouns that are avaliable in the abstract and use them to represent the different topis that are avaliable.